In [2]:
import gc
import os

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

import torch
import torch.optim as optim

In [4]:
model_checkpoint = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [9]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [10]:
test.drop(['Unnamed: 0'], axis=1, inplace=True)

In [11]:
test

,video_name,category_name,eng_videos,ru_videos,text_and_video,text,text_len
0,0.mp4,Новости и СМИ,A man and a woman are sitting at a table and a...,"Мужчина и женщина сидят за столом, а мужчина р...",Новости и СМИ<tab>Добро пожаловать! Вы находит...,Добро пожаловать! Вы находитесь в бюро добрых ...,1170
1,1.mp4,Развлечения,A man is showing how to make a pan of sausages...,"Мужчина показывает, как сделать кастрюлю из ша...",Развлечения<tab>Ты видел в жизни когда-нибудь ...,Ты видел в жизни когда-нибудь Киркорова? Да. Я...,1475
2,2.mp4,Развлечения,A woman is talking to a camera and then anothe...,"Женщина говорит о проекте, над которым она раб...","Развлечения<tab>Привет, вы смотрите True Detec...","Привет, вы смотрите True Detector, шоу в котор...",1560
3,3.mp4,Видеоигры,A group of people are sitting at a table and a...,"Группа людей сидит за столом, и женщина о чем-...",Видеоигры<tab>На серьезки? Ооо! Это зря! Два с...,На серьезки? Ооо! Это зря! Два стула… Мне 20 л...,1532
4,4.mp4,Развлечения,A man and a woman are talking while a man push...,"Мужчина и женщина разговаривают, в то время ка...",Развлечения<tab>ИНТРИГУЮЩАЯ МУЗЫКА Ну и где он...,ИНТРИГУЮЩАЯ МУЗЫКА Ну и где он? Мы сейчас его ...,1467
...,...,...,...,...,...,...,...
95,95.mp4,Развлечения,A group of people are on a soccer field and on...,"Группа людей находится на футбольном поле, и о...",Развлечения<tab>Всем здорова! Это самое адеква...,Всем здорова! Это самое адекватное по моим мер...,1098
96,96.mp4,Юмор,A man wearing sunglasses and sunglasses is wea...,Мужчина в солнцезащитных очках носит маску и ш...,Юмор<tab>а можно не быть таким дрищом привет г...,а можно не быть таким дрищом привет говна с то...,1065
97,97.mp4,Спорт,A group of men and women are playing a game of...,Группа мужчин и женщин играет в футбол перед г...,"Спорт<tab>Федос и Булат. Хорош, Федос! Смотри,...","Федос и Булат. Хорош, Федос! Смотри, я добрал ...",1376
98,98.mp4,Блоги,A group of men and women are sitting at a tabl...,Группа мужчин и женщин сидят за столом и говор...,Блоги<tab>Всем привет! С вами шоу Хашлама. Это...,"Всем привет! С вами шоу Хашлама. Это то шоу, г...",1475


In [14]:
def audio2submit(model, tokenizer, df, max_length=10, test_column='text_and_video'):
  summaries = []
  for row in df.iterrows():
      article_text = row[1][test_column]
      input_ids = tokenizer(
          ['summarize: ' + article_text],
          max_length=max_length,
          padding="max_length",
          truncation=True,
          return_tensors="pt",
      )["input_ids"]

      output_ids = model.generate(
          input_ids=input_ids,
          no_repeat_ngram_size=4
      )[0]
      summary = tokenizer.decode(output_ids, skip_special_tokens=True)
      summaries.append(summary)
  res_dict = {'video_name': [f'{i}.mp4' for i in range(100)], 'generated_description': summaries}
  res_df = pd.DataFrame(res_dict)
  res_df.to_csv('submission.csv')
  return summaries

In [15]:
summaries = audio2submit(model, tokenizer, test)

KeyboardInterrupt: ignored